In [ ]:
Save the circuit as a HDF5 file. We later upload the HDF5 into our Qiskit environments.


# Load Julia’s package manager interface
using Pkg

# Install Jupyter notebook integration for Julia
Pkg.add("IJulia")

# Install Julia’s built-in random number utilities
Pkg.add("Random")

# Install formatted string printing (similar to printf in C)
Pkg.add("Printf")


# Load the modules into the current session:

# OrderedCollections: e.g., OrderedDict for deterministic key ordering
using OrderedCollections

# HDF5: reading from and writing to HDF5 data files
using HDF5




"""
    package!(actual_circuit::Vector{Vector{ITensor}}) → Vector{Any}

Convert an entangling circuit (list of layers of ITensor gates) into plain Julia arrays
 suitable for numerical backend or storage.  Two‐qubit gates become 4×4 matrices, and
 single‐qubit boundary gates become 2×2 matrices.

# Arguments
- `actual_circuit::Vector{Vector{ITensor}}`  
  Depth‐D circuit: a vector of D layers, each containing N two‐site gates except the  
  last gate in each layer (a single‐site gate).

# Returns
- `packaged_layers::Vector{Any}`  
  A length‐D vector where each entry is a vector of N arrays:
  - For gates 1 through N−1: 4×4 matrices (reshaped from (2,2,2,2) tensors).
  - For gate N: a 2×2 matrix (single‐site gate).
"""
function package!(actual_circuit)
    D = length(actual_circuit)
    packaged_layers = Vector{Any}(undef, D)
    
    for d in 1:D
        layer = actual_circuit[d]
        N = length(layer)
        matrix_gates = Vector{Any}(undef, N)
        
        # Process the first N-1 gates as two-qubit gates.
        for n in 1:N-1
            gate = layer[n]
            # Identify output and input physical indices
            s1_out, s2_out, s1_in, s2_in = inds(gate, "Site")...
            # Extract the raw 4-index array and reshape to 4×4
            gate_array = Array(gate, (s1_out, s2_out, s1_in, s2_in))
            matrix_gates[n] = reshape(gate_array, (4, 4))
        end
        
        # Process the final gate in the layer as a single-qubit gate.
        gate = layer[N]
        s1_out, s1_in = inds(gate, "Site")...
        gate_array = Array(gate, (s1_out, s1_in))
        matrix_gates[N] = reshape(gate_array, (2, 2))
        
        packaged_layers[d] = matrix_gates
    end
    
    return packaged_layers
end

"""
    savepackage!(filename::String, packaged_layers::Vector{Any})

Write the packaged circuit layers to an HDF5 file.  Each gate is stored under a dataset
name `"layer<d>_gate<g>"`.

# Arguments
- `filename::String`  
  Path to the output `.h5` file.
- `packaged_layers::Vector{Any}`  
  Output from `package!`, a length‐D vector of gate‐matrix vectors.
"""
function savepackage!(filename::String, packaged_layers::Vector{Any})
    h5open(filename, "w") do file
        for (layer_idx, layer) in enumerate(packaged_layers)
            for (gate_idx, gate_mat) in enumerate(layer)
                key = "layer$(layer_idx)_gate$(gate_idx)"
                write(file, key, gate_mat)
            end
        end
    end
end

# ————————————————————————————————————————————————
# Example Usage: generate, package, and save an entangling circuit

# (1) Prepare MPS and product state; build the entangling circuit
ψ_orig       = copy(psi_non_truncated)
product_state = copy(product_state)
D = 1  # choose circuit depth
circuit_layers = deep!(ψ_orig, D)
actual_circuit = reverse_deep!(circuit_layers)

# (2) Convert the ITensor gates into plain matrices
packaged_layers = package!(actual_circuit)

# (3) Determine circuit dimensions
N = length(actual_circuit[1])  # number of gates per layer (≈ number of sites)
D = length(actual_circuit)     # circuit depth

# (4) Save to HDF5 with descriptive filename
filename = "gates_N$(N)_D$(D).h5"
savepackage!(filename, packaged_layers)
println("Saved circuit for N = $N and depth D = $D to ", filename)

In [ ]:
using OrderedCollections, HDF5

"""
    generate_circuits_range(ψ0::MPS, prod0::MPS; depths=1:10) → OrderedDict{Int, Vector{Vector{ITensor}}}

Build entangling circuits of various depths for a given target MPS `ψ0` and an initial product state `prod0`.

# Arguments
- `ψ0::MPS`  
  The target MPS to reconstruct (e.g., the ground state).  
- `prod0::MPS`  
  The simple product-state MPS (e.g., all spins down).  
- `depths`  
  An iterable of integer depths at which to generate circuits (default `1:10`).

# Returns
- `circuits::OrderedDict{Int, Vector{Vector{ITensor}}}`  
  Maps each depth `d` to the entangling circuit for that depth, represented as a vector of layers,
  each layer itself a vector of ITensor gates.
"""
function generate_circuits_range(ψ0, prod0; depths=1:10)
    depths_vec = sort(collect(depths))                  # ensure depths are in ascending order
    circuits = OrderedDict{Int, Vector{Vector{ITensor}}}()

    for d in depths_vec
        # Work on fresh copies each time
        ψ    = copy(ψ0)
        prod = copy(prod0)
        # Build the disentangling circuit of depth d
        layers = deep!(ψ, d)
        # Invert it to get the entangling circuit
        circuits[d] = reverse_deep!(layers)
    end

    return circuits
end




"""
    package!(actual_circuit::Vector{Vector{ITensor}}) → Vector{Any}

Convert a single-depth entangling circuit into a sequence of plain Julia arrays.
Two-site gates become 4×4 matrices; the final single-site gate in each layer becomes a 2×2 matrix.

# Arguments
- `actual_circuit`  
  A vector of ITensor‐gate layers for one particular depth.

# Returns
- `packaged::Vector{Any}`  
  A vector of length = circuit depth D, where each entry is a vector of N matrices:
  - entries 1..N-1 are 4×4 gate matrices,
  - entry N is a 2×2 single‐site gate matrix.
"""
function package!(actual_circuit::Vector{<:Any})
    D = length(actual_circuit)
    packaged = Vector{Any}(undef, D)

    for d in 1:D
        layer = actual_circuit[d]
        N = length(layer)
        mats = Vector{Any}(undef, N)

        # Process the two-qubit gates
        for n in 1:N-1
            gate = layer[n]
            o1, o2, i1, i2 = inds(gate, "Site")
            A = Array(gate, (o1, o2, i1, i2))  # raw 4-index tensor
            mats[n] = reshape(A, 4, 4)         # flatten to 4×4 matrix
        end

        # Process the final single-qubit gate
        gate = layer[N]
        o, i = inds(gate, "Site")
        A = Array(gate, (o, i))               # raw 2-index tensor
        mats[N] = reshape(A, 2, 2)            # as 2×2 matrix

        packaged[d] = mats
    end

    return packaged
end





"""
    savepackage!(filename::String, packaged_layers::Vector{Any})

Save a packaged circuit (from `package!`) into an HDF5 file.  Each gate is stored under
a dataset name `"layer<d>_gate<g>"`.

# Arguments
- `filename::String`  
  The output `.h5` file path.  
- `packaged_layers::Vector{Any}`  
  The packaged circuit matrices from `package!`.
"""
function savepackage!(filename::String, packaged_layers::Vector{Any})
    h5open(filename, "w") do file
        for (li, layer_mats) in enumerate(packaged_layers)
            for (gi, mat) in enumerate(layer_mats)
                ds = "layer$(li)_gate$(gi)"
                write(file, ds, mat)
            end
        end
    end
end






"""
    save_all_depths!(
        ψ0::MPS,
        prod0::MPS;
        depths=1:10,
        out_prefix="gates_N"
    ) → Nothing

Generate and save entangling circuits for multiple depths in one go.
For each depth `d`, packages the circuit and writes it to
`"<out_prefix><N>_D<d>.h5"`.

# Arguments
- `ψ0::MPS`  
  Target MPS to reconstruct.  
- `prod0::MPS`  
  Initial product-state MPS.  
- `depths`  
  Iterable of depths (default `1:10`).  
- `out_prefix::String`  
  Prefix for output filenames (default `"gates_N"`; the system size `N` is appended).
"""
function save_all_depths!(
        ψ0, prod0;
        depths=1:10,
        out_prefix="gates_N"
    )

    # Generate all circuits by depth
    circuits_by_depth = generate_circuits_range(ψ0, prod0; depths=depths)

    # Loop and save each one
    for (d, circuit_layers) in circuits_by_depth
        mats = package!(circuit_layers)                # convert to arrays
        N = length(circuit_layers[1])                  # number of gates per layer

        # Construct filename encoding N and d
        fname = "$(out_prefix)$(N)_D$(d).h5"
        savepackage!(fname, mats)                      # write to disk
        println("Saved depth $d (N=$N) → $fname")
    end
end

# ————————————————————————————————————————————————
# Run for N-site system over depths 1 to 10:
save_all_depths!(copy(psi_non_truncated), copy(product_state); depths=1:10)
